In [1]:
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV


In [2]:
train = pd.read_csv('train.csv', index_col=0).sample(frac=1)
y_train = train['correct']
X_train = train.drop(columns=['name', 'firstChar', 'group', 'correct'])

test = pd.read_csv('test.csv', index_col=0).sample(frac=1)
y_test = test['correct']
X_test = test.drop(columns=['name', 'firstChar', 'group', 'correct'])

In [3]:
def show_rank_metrics(df_group: pd.DataFrame, df_proba: pd.DataFrame, df_true: pd.DataFrame):
    df_metric = pd.concat([df_group, df_proba, df_true], axis=1)
    df_metric = df_metric.sort_values(by=['group', 'proba', 'correct'], ascending=[True, False, True])
    print(df_metric.shape)
    positions = []
    cur_group = -1
    cur_pos = 1
    for row in df_metric.itertuples():
        cur_pos += 1
        if row.group != cur_group:
            cur_pos = 1
            cur_group = row.group
        if row.correct == 1:
            positions.append(cur_pos)
    print(f'\nmean = {np.mean(positions)}\n')

    count = [0] * 9
    for p in positions:
        count[p] += 1
    acc = 0
    sum_all = sum(count)
    for i, c in enumerate(count):
        acc += c
        print(f'top{i} = {acc / sum_all}')

In [4]:
# Initialize CatBoostClassifier
model = CatBoostClassifier(verbose=False)
model.fit(X_train, y_train)

In [5]:
model.predict_proba(X_test)

array([[0.98704528, 0.01295472],
       [0.99221775, 0.00778225],
       [0.90575396, 0.09424604],
       ...,
       [0.87940448, 0.12059552],
       [0.97178878, 0.02821122],
       [0.81845443, 0.18154557]])

In [6]:
show_rank_metrics(
    df_group=pd.DataFrame(data=test['group'].tolist(), columns=['group']),
    df_proba=pd.DataFrame(data=model.predict_proba(X_test)[:,1], columns=['proba']),
    df_true=pd.DataFrame(data=y_test.tolist(), columns=['correct'])
)

(107568, 3)

mean = 2.051836977539789

top0 = 0.0
top1 = 0.5980217164956121
top2 = 0.7339729287520452
top3 = 0.8222519708463484
top4 = 0.8908969210174029
top5 = 0.9384947196192176
top6 = 0.9729287520452179
top7 = 0.9915960136843671
top8 = 1.0


In [7]:
model.save_model("kek")